In [ ]:
import torch as th
import torch
from typing import Optional

In [ ]:
def b_spline_first(x: torch.Tensor, i: int, k: int) -> torch.Tensor:
    initial_i = i
    n = x.size(0)

    def __nodes(_i: int) -> float:
        return (_i + k // 2 - k % 2) / (n + (k // 2 - k % 2) * 2)

    def __b_spline(_x: torch.Tensor, _i: int, _k: int) -> torch.Tensor:
        if _k == 0:
            return torch.logical_and(
                torch.ge(_x[initial_i], __nodes(_i)),
                torch.lt(_x[initial_i], __nodes(_i + 1)),
            ).to(torch.float)

        return __b_spline(_x, _i, _k - 1) * (_x[initial_i] - __nodes(_i)) / (
            __nodes(_i + _k) - __nodes(_i)
        ) + __b_spline(_x, _i + 1, _k - 1) * (
            __nodes(_i + _k + 1) - _x[initial_i]
        ) / (
            __nodes(_i + _k + 1) - __nodes(_i + 1)
        )

    return __b_spline(x, i, k)

In [ ]:
def b_spline(x: th.Tensor, k: int) -> th.Tensor:
    batch_size, n = x.size()[:2]

    offset = k // 2 - k % 2

    def __nodes(_i: th.Tensor) -> th.Tensor:
        return (_i + offset) / (n + offset * 2)

    i_s = th.arange(n, device=x.device).unsqueeze(0)

    def __b_spline(curr_i_s: th.Tensor, curr_k: int) -> th.Tensor:
        if curr_k == 0:
            return th.logical_and(
                th.ge(x, __nodes(curr_i_s)), th.lt(x, __nodes(curr_i_s + 1))
            ).to(th.float)

        return __b_spline(curr_i_s, curr_k - 1) * (x - __nodes(curr_i_s)) / (
            __nodes(curr_i_s + curr_k) - __nodes(curr_i_s)
        ) + __b_spline(curr_i_s + 1, curr_k - 1) * (
            __nodes(curr_i_s + curr_k + 1) - x
        ) / (
            __nodes(curr_i_s + curr_k + 1) - __nodes(curr_i_s + 1)
        )

    return __b_spline(i_s, k)

In [ ]:
x_in = torch.randn(10)

In [ ]:
print(
    "["
    + ", ".join(
        [str(b_spline_first(x_in, i, 3).float()) for i in range(x_in.size(0))]
    )
    + "]"
)

In [ ]:
print(b_spline(x_in.unsqueeze(0), 3))